#### 데이터 전처리 과정

In [1]:
# konlpy 설치

! pip install konlpy

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os

path = "C:/Users/82106/Desktop/2024-1/머신러닝/project"
os.getcwd()
os.chdir(path)
os.getcwd()

'C:\\Users\\82106\\Desktop\\2024-1\\머신러닝\\project'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [8]:
# 쇼핑 리뷰 데이터 불러오기

df = pd.read_table('naver_shopping.txt', names=['ratings', 'reviews'])
df

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~


In [9]:
# 중복 데이터 제거

df.drop_duplicates('reviews',keep='first')
df.shape

(200000, 2)

#### 텍스트 전처리

In [13]:
#별점으로 라벨링 : 별점이 4을 넘으면 긍정, 이하는 부정으로 분류

def star_evaluate(rating):
  if rating >=4:
    return 1
  else:
    return 0
  
df['label'] = df['ratings'].apply(star_evaluate)
df

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1
...,...,...,...
199995,2,장마라그런가!!! 달지않아요,0
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...,1
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요,1
199998,5,넘이쁘고 쎄련되보이네요~,1


In [14]:
#정규표현식 사용하여 한글 제외 모두 제거
df['reviews'] = df['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['reviews'].replace('', np.nan, inplace=True)
df.head()

C:\Users\82106\AppData\Local\Temp\ipykernel_19800\2660665286.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews'].replace('', np.nan, inplace=True)


,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


#### 토큰화

In [15]:
# okt
from konlpy.tag import Okt
okt = Okt()

In [16]:
# 불용어 파일 읽어오기
with open('updated_stop_words.txt', 'r', encoding='utf-8') as file:
    stopwords_text = file.readlines()

# 읽어온 텍스트를 파이썬 리스트로 변환하고 불필요한 공백 및 개행 문자 제거
stopwords = [word.strip() for word in stopwords_text if word.strip()]

print(stopwords[:10])

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']


In [17]:
# 토큰화

df['tokenized'] = df['reviews'].apply(okt.morphs)
df['tokenized'] = df['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [18]:
df.head()

,ratings,reviews,label,tokenized
0,5,배공빠르고 굿,1,"[배공, 빠르고, 굿]"
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,"[택배, 엉망, 밑, 층, 놔두고가고]"
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1,"[좋아요, 바지, 좋아서, 구매, 가격, 대박, 바느질, 엉, 성하긴, 편하고, 가..."
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,"[선물, 빨리, 받아서, 전달, 했어야, 상품, 이었는데, 머그컵, 당황, 전화했더..."
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,"[민트, 색상, 예뻐요, 옆, 손잡이, 도로, 사용, 되네요]"


In [19]:
# 데이터프레임을 CSV 파일로 저장합니다.
df.to_csv('naver_shopping_tokenized1245.csv', index=False, encoding='utf-8-sig')